### Set up a Python virtual environment in Visual Studio Code

1. Open the Command Palette (Ctrl+Shift+P).
1. Search for **Python: Create Environment**.
1. Select **Venv**.
1. Select a Python interpreter. Choose 3.10 or later.

It can take a minute to set up. If you run into problems, see [Python environments in VS Code](https://code.visualstudio.com/docs/python/environments).

### Install packages

In [ ]:
! pip install -r phi-rag-requirements.txt --quiet

### Provision the sample

This sample uses [`azd`](https://learn.microsoft.com/azure/developer/azure-developer-cli/), a bicep template, and a custom post-provision hook to provision the sample. The sample uses role-based authentication using your identity for authentication, and does not use any API keys.

1. Open a PowerShell command prompt in the multimodal-embeddings folder.

1. Run `azd config set defaults.subscription <yourSubscriptionID>` to set the subscription if you have multiple Azure subscriptions.
1. If you want to use an existing resource, set the corresponding `azd` environment variables before deployment:
   1. Existing Search resource:
      1. `azd env set AZURE_SEARCH_SERVICE <your-search-service-name>`
      1. `azd env set AZURE_SEARCH_SERVICE_LOCATION <your-search-service-location>`
      1. `azd env set AZURE_SEARCH_SERVICE_RESOURCE_GROUP <your-search-service-resource-group>`
      1. `azd env set AZURE_SEARCH_SERVICE_SKU <your-search-service-sku>`
      1. `azd env set AZURE_SEARCH_SERVICE_SEMANTIC_RANKER <your-semantic-ranker-sku>`
   1. Existing AI Services resource:
      1. `azd env set AZURE_AI_SERVICES_ACCOUNT <your-ai-services-account-name>`
      1. `azd env set AZURE_AI_SERVICES_LOCATION <your-ai-services-account-location>`
      1. `azd env set AZURE_AI_SERVICES_RESOURCE_GROUP <your-ai-services-resource-group>`
   1. Existing Storage resource:
      1. `azd env set AZURE_STORAGE_ACCOUNT <your-storage-account-name>`
      1. `azd env set AZURE_STORAGE_ACCOUNT_LOCATION <your-storage-account-location>`
      1. `azd env set AZURE_STORAGE_ACCOUNT_RESOURCE_GROUP <your-storage-account-resource-group>`
1. Run `azd up`.
   1. Enter a development environment name.
   1. Enter a region for the function app. Choose a region that provides the Image Retrieval API: `East US`, `France Central`, `Korea Central`, `North Europe`, `Southeast Asia`, `West Europe`, `West US`.


If you aren't prompted for an environment or region, retry `azd up -e` to specify a new environment and region.

This step takes several minutes to complete.

### Retrieve environment variables after provisioning

The included `azd` bicep template saves all required environment variables for the notebook automatically.

In [1]:
# Load all environment variables from the azd deployment
import subprocess
from io import StringIO
from dotenv import load_dotenv
import os
result = subprocess.run("azd env get-values", stdout=subprocess.PIPE, cwd=os.getcwd())
load_dotenv(override=True, stream=StringIO(result.stdout.decode("utf-8")))

True

## Validate the indexer has completed successfully

An indexer runs in the background to process all the sample images. Validate that it has completed without any errors before trying to search the sample index

In [2]:
from azure.search.documents.indexes import SearchIndexerClient
from azure.identity import DefaultAzureCredential

search_indexer_client = SearchIndexerClient(endpoint=os.getenv("AZURE_SEARCH_ENDPOINT"), credential=DefaultAzureCredential())
status = search_indexer_client.get_indexer_status(name=os.getenv("AZURE_SEARCH_INDEXER"))
print(f"Status: {status.last_result.status}, Items Processed: {status.last_result.item_count}, Items Failed: {status.last_result.failed_item_count}")

Status: success, Items Processed: 0, Items Failed: 0


## Get Phi 3 scoring URI and key

Connect to the Azure AI Project and fetch the deployed Phi scoring uri and key for access

In [3]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

workspace_ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id=os.getenv("AZURE_SUBSCRIPTION_ID"),
    resource_group_name=os.getenv("AZURE_RESOURCE_GROUP"),
    workspace_name=os.getenv("AZUREAI_PROJECT_NAME")
)

phi3_url = workspace_ml_client.online_endpoints.get(name=os.getenv("AZUREAI_ONLINE_ENDPOINT_NAME")).scoring_uri
phi3_key = workspace_ml_client.online_endpoints.get_keys(name=os.getenv("AZUREAI_ONLINE_ENDPOINT_NAME")).primary_key

# Setup search parameters for grounding the conversation on your documents

Change the various types of searches to change how chunks are fetched from the index

In [4]:
k=50
search_type="hybrid"
use_semantic_reranker=True
sources_to_include=5

## Chat using Phi 3

Start the conversation using Phi 3

In [5]:
import azure.identity.aio
from lib.chat import ChatThread, create_search_client, SearchType

chat_thread = ChatThread()

async with azure.identity.aio.DefaultAzureCredential() as credential, create_search_client(credential) as search_client:
    await chat_thread.append_grounded_message(
        search_client=search_client,
        query="What is included in my Northwind Health Plus plan that is not in standard? Answer concisely",
        search_type=SearchType(search_type),
        use_semantic_reranker=use_semantic_reranker,
        sources_to_include=sources_to_include,
        k=k)
    await chat_thread.get_phi3_response(endpoint_scoring_uri=phi3_url, endpoint_authorization="Bearer " + phi3_key, deployment=os.getenv("AZUREAI_DEPLOYMENT_NAME"))

print(chat_thread.get_last_message()["content"])

{'output': '\nYour Northwind Health Plus plan includes coverage for counseling, psychotherapy, and \n\nother treatment related to mental health and substance abuse. This means that services \n\nprovided by independent contractors may be covered under the Northwind Health Plus plan, \n\nprovided that the service is medically necessary. \n\nServices provided by independent contractors that are not medically necessary, such as \n\nmassage therapy or acupuncture, are not covered under the Northwind Health Plus plan.\n\n\nOut-of-Network Services  \n\nNorthwind'}

Your Northwind Health Plus plan includes coverage for counseling, psychotherapy, and 

other treatment related to mental health and substance abuse. This means that services 

provided by independent contractors may be covered under the Northwind Health Plus plan, 

provided that the service is medically necessary. 

Services provided by independent contractors that are not medically necessary, such as 

massage therapy or acupunct

## (Optional) Chat using GPT4 or GPT-35-Turbo

Continue your conversation using OpenAI models

In [7]:
from lib.chat import ChatThread, create_search_client, create_openai_client, SearchType

chat_deployment = os.getenv("AZURE_OPENAI_GPT4_DEPLOYMENT_NAME") # replace with AZURE_OPENAI_GPT35TURBO_DEPLOYMENT_NAME to use gpt-35-turbo

async with azure.identity.aio.DefaultAzureCredential() as credential, create_search_client(credential) as search_client, create_openai_client(credential) as openai_client:
    await chat_thread.append_grounded_message(
        search_client=search_client,
        query="What is included in my Northwind Health Plus plan that is not in standard?",
        search_type=SearchType(search_type),
        use_semantic_reranker=use_semantic_reranker,
        sources_to_include=sources_to_include,
        k=k)
    await chat_thread.get_openai_response(openai_client=openai_client, model=chat_deployment)

print(chat_thread.get_last_message()["content"])


The Northwind Health Plus plan includes several benefits not found in the Northwind Standard plan:

1. **Coverage for Services by Independent Contractors:** Northwind Health Plus includes coverage for healthcare services provided by independent contractors, provided that the service is medically necessary [Northwind_Health_Plus_Benefits_Details.pdf].

2. **Out-of-Network Emergency Services:** Northwind Health Plus pays for emergency services both in-network and out-of-network, with certain criteria [Northwind_Health_Plus_Benefits_Details.pdf].

3. **Coverage for Hospital Stays, X-rays, and a wider range of Prescription Drug Coverage:** Unlike Northwind Standard, Northwind Health Plus covers hospital stays, X-rays, and offers a more extensive range of prescription drug coverage [Benefit_Options.pdf].

4. **Personal Health Support Programs:** Northwind Health Plus offers personal health support programs including case management, disease management, and wellness programs to help members 

## Inspect sources used by the Language Model

Check which sources were used to ground the response

In [4]:
chat_thread.get_last_message_sources()

[{'metadata_storage_name': 'Northwind_Standard_Benefits_Details.pdf',
  'content': 'includes counselling, psychotherapy, \n\nand other treatments related to mental health and substance abuse.  \n\n\n\nOut-of-Network Services: The Northwind Standard plan does not cover any services that are \n\nprovided by a provider that is not part of the Northwind Health network. This includes \n\ndoctors, hospitals, and other healthcare providers who are not part of the Northwind Health \n\nnetwork.  \n\nTips \n\nWhen selecting a healthcare plan, it is important to be aware of the exclusions in the plan. \n\nHere are some tips to help you understand the exclusions in the Northwind Standard plan:  \n\n1. Understand the types of services that are not covered by the Northwind Standard plan. Be \n\nsure to familiarize yourself with the list of exclusions and make sure that any services you \n\nmight require are covered.  \n\n2. If you require emergency services, be sure to check with your provider to se

### 